In [12]:
import json
import web3

#from web3 import Web3
from web3 import Web3, HTTPProvider
from solc import compile_source
from web3.contract import ConciseContract

import pandas as pd

data = pd.read_csv("Llaves.csv") 

public_keys = data['Llave Publica']
private_keys = data['Llave Privada']

w3 = Web3(HTTPProvider("https://rinkeby.infura.io/v3/4c0ec7f1412a489d91e1934c66ebf5b1"))

## Transfer Ether

In [32]:
acct = w3.eth.account.privateKeyToAccount(private_keys[0])
totalSupply = w3.eth.getBalance(public_keys[0])
print('Total Supply = '+str(totalSupply))
balance = 1
for address in public_keys[1:]:
    
    signed_txn = w3.eth.account.signTransaction({
        'from': acct.address,
        'nonce': w3.eth.getTransactionCount(acct.address),
        'gas': 3000000,
        'to':address,
        'gasPrice': w3.toWei('1.4', 'gwei'),
        'value':w3.toWei(1,'ether')},
        private_keys[0])
    
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

Total Supply = 23643331852400000000


## Coops

In [7]:
# Solidity source code
Coops_source_code = '''

pragma solidity ^0.4.24;

//import "./Owned.sol";
//import "./CrowdFunding.sol";

// ----------------------------------------------------------------------------
// 'Coops' token contract
//
// Deployed to : 0xdD870fA1b7C4700F2BD7f44238821C26f7392148
// Symbol      : Coops
// Name        : Coops Token
// Total supply: 100000000
// Decimals    : 18
//
// Enjoy.
//
// (c) by Moritz Neto with BokkyPooBah / Bok Consulting Pty Ltd Au 2017. The MIT Licence.
// ----------------------------------------------------------------------------


// ----------------------------------------------------------------------------
// Safe maths
// ----------------------------------------------------------------------------
contract SafeMath {
    function safeAdd(uint a, uint b) public pure returns (uint c) {
        c = a + b;
        require(c >= a);
    }
    function safeSub(uint a, uint b) public pure returns (uint c) {
        require(b <= a);
        c = a - b;
    }
    function safeMul(uint a, uint b) public pure returns (uint c) {
        c = a * b;
        require(a == 0 || c / a == b);
    }
    function safeDiv(uint a, uint b) public pure returns (uint c) {
        require(b > 0);
        c = a / b;
    }
}


// ----------------------------------------------------------------------------
// ERC Token Standard #20 Interface
// https://github.com/ethereum/EIPs/blob/master/EIPS/eip-20-token-standard.md
// ----------------------------------------------------------------------------
contract ERC20Interface {
    function totalSupply() public constant returns (uint);
    function balanceOf(address tokenOwner) public constant returns (uint balance);
    function allowance(address tokenOwner, address spender) public constant returns (uint remaining);
    function transfer(address to, uint tokens) public returns (bool success);
    function approve(address spender, uint tokens) public returns (bool success);
    function transferFrom(address from, address to, uint tokens) public returns (bool success);

    event Transfer(address indexed from, address indexed to, uint tokens);
    event Approval(address indexed tokenOwner, address indexed spender, uint tokens);
}


// ----------------------------------------------------------------------------
// Contract function to receive approval and execute function in one call
//
// Borrowed from MiniMeToken
// ----------------------------------------------------------------------------
contract ApproveAndCallFallBack {
    function receiveApproval(address from, uint256 tokens, address token, bytes data) public;
}

// ----------------------------------------------------------------------------
// Owned contract
// ----------------------------------------------------------------------------
contract Owned {
    address public owner;
    address public newOwner;

    event OwnershipTransferred(address indexed _from, address indexed _to);

    constructor() public {
        owner = msg.sender;
    }

    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }

    function transferOwnership(address _newOwner) public onlyOwner {
        newOwner = _newOwner;
    }
    function acceptOwnership() public {
        require(msg.sender == newOwner);
        emit OwnershipTransferred(owner, newOwner);
        owner = newOwner;
        newOwner = address(0);
    }
}

contract CrowdFunding is Owned{
    
    address public beneficiary;
    uint public fundingGoal;
    uint public amountRaised;
    uint public deadline;
    uint numFunders = 0;
    mapping(address => uint256) public balanceOf;
    //mapping(uint => address) Funders;
    //mapping(uint => uint) Funds;
    address[] Funders;
    uint[] Funds;
    bool fundingGoalReached = false;
    bool crowdsaleClosed = true;

    event GoalReached(address recipient, uint totalAmountRaised);
    event FundTransfer(address backer, uint amount, bool isContribution);
    
    address Ccontract = 0x3F5C926EC1BbDBD6C54DBdAAde5573D8e0205121;
    Coops C = Coops(Ccontract);

    /**
     * Constructor function
     *
     * Setup the owner
     */
     
     
    function Crowdsale (
        address _beneficiary,
        uint _fundingGoal,
        uint durationInMinutes
    ) onlyOwner 
      public {
        require(crowdsaleClosed);
        beneficiary = _beneficiary;
        fundingGoal = _fundingGoal;  //InERC20;
        deadline = now + durationInMinutes * 2 seconds;
        crowdsaleClosed = false;
        amountRaised = 0;
        for (uint i = 0; i < numFunders; i++){
            balanceOf[Funders[i]] = 0;
        }
        numFunders = 0;
    }

    /**
     * Fallback function
     *
     * The function without name is the default function that is called whenever anyone sends funds to a contract
     */
    /*  DONATIONS IN ETHER
    function () payable public {
        require(!crowdsaleClosed);
        uint amount = msg.value; 
        balanceOf[msg.sender] += amount;
        amountRaised += amount;
        emit FundTransfer(msg.sender, amount, true);
        //address(this);
    }*/
    
    function receiveFund
    (address _sender,
    uint256 _tokens,
    Coops _tokenContract)
    public{
      require(_tokenContract == C);
      require(!crowdsaleClosed);
      if(amountRaised > fundingGoal) revert();
      require(C.transferFrom(_sender, this, _tokens));
      balanceOf[_sender] += _tokens;
      amountRaised += _tokens;
      //Funders[numFunders] = _sender;
      //Funds[numFunders] = _tokens;
      Funders.push(_sender);
      Funds.push(_tokens);
      numFunders += 1;
      emit FundTransfer(msg.sender, _tokens, true);
    }

    modifier afterDeadline() { if (now >= deadline) _; }

    /**
     * Check if goal was reached
     *
     * Checks if the goal or time limit has been reached and ends the campaign
     */
    function checkGoalReached() public afterDeadline {
        //if (amountRaised >= fundingGoal){
        if (C.balanceOf(this) >= fundingGoal){
            fundingGoalReached = true;
            emit GoalReached(beneficiary, amountRaised);
        }
        crowdsaleClosed = true;
    }


    /**
     * Withdraw the funds
     *
     * Checks to see if goal or time limit has been reached, and if so, and the funding goal was reached,
     * sends the entire amount to the beneficiary. If goal was not reached, each contributor can withdraw
     * the amount they contributed.
     */
    function safeWithdrawal() public afterDeadline {
        if (!fundingGoalReached) {
            uint amount = balanceOf[msg.sender];
            balanceOf[msg.sender] = 0;
            if (amount > 0) {
                if (C.transfer(msg.sender, amount)){
                   emit FundTransfer(msg.sender, amount, false);
                } else {
                    balanceOf[msg.sender] = amount;
                }
            }
        }

        if (fundingGoalReached && beneficiary == msg.sender) {
            if (C.transfer(beneficiary, amountRaised)) {
               emit FundTransfer(beneficiary, amountRaised, false);
                for (uint i = 0; i < numFunders; i++){
                    balanceOf[Funders[i]] = 0;
                }
                C.NewProject(beneficiary,
                    Funders, 
                    Funds, 
                    amountRaised);
            } else {
                //If we fail to send the funds to beneficiary, unlock funders balance
                fundingGoalReached = false;
            }
            //if (C.transfer(beneficiary, amountRaised)) revert();
        }
    }
}




// ----------------------------------------------------------------------------
// ERC20 Token, with the addition of symbol, name and decimals and assisted
// token transfers
// ----------------------------------------------------------------------------



contract Coops is ERC20Interface, Owned, SafeMath {
    
    struct Project{
        address owner;
        address[] funders;
        uint[] amounts;
        uint totalPrice;
        uint ID;
    }
    
    
    string public symbol;
    string public  name;
    
    
    uint8 public decimals;
    uint public _totalSupply;
    
    uint numProjects = 0;
    uint kwh_price = 100;

    mapping(address => uint) balances;
    mapping(address => mapping(address => uint)) allowed;
    mapping(address => mapping(address => uint)) private Debts;
    mapping(address => uint) private totalDebts;
    mapping(uint => Project) private Projects; 

    // ------------------------------------------------------------------------
    // Constructor
    // ------------------------------------------------------------------------
    constructor() public {
        symbol = "Coops";
        name = "Coops Token";
        decimals = 18;
        _totalSupply = 100000000;
        balances[owner] = _totalSupply;
        emit Transfer(address(0), owner, _totalSupply);
    }


    // ------------------------------------------------------------------------
    // Total supply
    // ------------------------------------------------------------------------
    function totalSupply() public constant returns (uint) {
        return _totalSupply  - balances[address(0)];
    }


    // ------------------------------------------------------------------------
    // Get the token balance for account tokenOwner
    // ------------------------------------------------------------------------
    function balanceOf(address tokenOwner) public constant returns (uint balance) {
        return balances[tokenOwner];
    }
    
    function getDebt(address debtor, address creditter) 
    public
    constant
    returns(uint debt)
    {
    return (Debts[debtor][creditter]);
    }
    
    function getTotalDebt(address debtor) 
    public
    constant
    returns(uint debt)
    {
    return (totalDebts[debtor]);
    }


    // ------------------------------------------------------------------------
    // Transfer the balance from token owner's account to to account
    // - Owner's account must have sufficient balance to transfer
    // - 0 value transfers are allowed
    // ------------------------------------------------------------------------
    function transfer(address to, uint tokens) public returns (bool success) {
        balances[msg.sender] = safeSub(balances[msg.sender], tokens);
        balances[to] = safeAdd(balances[to], tokens);
        emit Transfer(msg.sender, to, tokens);
        return true;
    }

    function Pay(address creditter, uint amount) 
    public
    returns(bool success) {
        balances[msg.sender] = safeSub(balances[msg.sender], amount);
        //Debts[debtor][creditter] -= amount;
        Debts[msg.sender][creditter] = safeSub(Debts[msg.sender][creditter],amount);
        //totalDebts[debtor] -= amount;
        totalDebts[msg.sender] = safeSub(totalDebts[msg.sender],amount);
        balances[creditter] = safeAdd(balances[creditter], amount);
        return true;
    } 

    // ------------------------------------------------------------------------
    // Token owner can approve for spender to transferFrom(...) tokens
    // from the token owner's account
    //
    // https://github.com/ethereum/EIPs/blob/master/EIPS/eip-20-token-standard.md
    // recommends that there are no checks for the approval double-spend attack
    // as this should be implemented in user interfaces 
    // ------------------------------------------------------------------------
    function approve(address spender, uint tokens) public returns (bool success) {
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        return true;
    }
    
    function addDebt(address debtor, address creditter, uint amount) 
    public
    returns(bool success) 
  {
    //Debts[debtor][creditter] += amount;
    Debts[debtor][creditter] = safeAdd(Debts[debtor][creditter], amount);
    //totalDebts[debtor] += amount;
    totalDebts[debtor] = safeAdd(totalDebts[debtor], amount);
    return true;
  }


    // ------------------------------------------------------------------------
    // Transfer tokens from the from account to the to account
    // 
    // The calling account must already have sufficient tokens approve(...)-d
    // for spending from the from account and
    // - From account must have sufficient balance to transfer
    // - Spender must have sufficient allowance to transfer
    // - 0 value transfers are allowed
    // ------------------------------------------------------------------------
    function transferFrom(address from, address to, uint tokens) public returns (bool success) {
        balances[from] = safeSub(balances[from], tokens);
        allowed[from][msg.sender] = safeSub(allowed[from][msg.sender], tokens);
        balances[to] = safeAdd(balances[to], tokens);
        emit Transfer(from, to, tokens);
        return true;
    }

    // ------------------------------------------------------------------------
    // Returns the amount of tokens approved by the owner that can be
    // transferred to the spender's account
    // ------------------------------------------------------------------------
    function allowance(address tokenOwner, address spender) public constant returns (uint remaining) {
        return allowed[tokenOwner][spender];
    }


    // ------------------------------------------------------------------------
    // Token owner can approve for spender to transferFrom(...) tokens
    // from the token owner's account. The spender contract function
    // receiveApproval(...) is then executed
    // ------------------------------------------------------------------------
    /*function approveAndCall(address spender, uint tokens, bytes data) public returns (bool success) {
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        ApproveAndCallFallBack(spender).receiveApproval(msg.sender, tokens, this, data);
        return true;
    }*/
    function Fund(address spender, uint tokens) public returns (bool success) {
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        //ApproveAndCallFallBack(spender).receiveApproval(msg.sender, tokens, this, data);
        //CrowdFunding F = CrowdFunding(Fcontract)
        CrowdFunding(spender).receiveFund(
            msg.sender, 
            tokens, 
            this);
        return true;
    }


    // ------------------------------------------------------------------------
    // Don't accept ETH
    // ------------------------------------------------------------------------
    function () public payable {
        revert();
    }


    // ------------------------------------------------------------------------
    // Owner can transfer out any accidentally sent ERC20 tokens
    // ------------------------------------------------------------------------
    function transferAnyERC20Token(address tokenAddress, uint tokens) public onlyOwner returns (bool success) {
        return ERC20Interface(tokenAddress).transfer(owner, tokens);
    }
    
    // ------------------------------------------------------------------------
    // Projects
    // ------------------------------------------------------------------------
    function NewProject(
        address owner,
        address[] funders, 
        uint[] amounts, 
        uint totalPrice) 
        public {
        //require(canCreate[msg.sender] == true);
        //require(D.getTotalDebt(owner) == 0);
        Projects[numProjects].owner = owner;
        Projects[numProjects].funders = funders;
        Projects[numProjects].amounts = amounts;
        Projects[numProjects].totalPrice = totalPrice;
        Projects[numProjects].ID = numProjects;
        numProjects += 1;
    }
    
    function Generate(uint kwh, uint ID) 
      public 
      {
        require(Projects[ID].owner == msg.sender);
        uint amount = kwh*kwh_price;
        uint amount_i;
        for (uint i = 0; i < (Projects[ID].funders).length; i++){
          amount_i = amount*(Projects[ID].amounts[i])/(Projects[ID].totalPrice);
          addDebt(msg.sender, (Projects[ID].funders[i]), amount_i);
        }
    }
}
'''

Coops_compiled_sol = compile_source(Coops_source_code,  import_remappings=['=./', '-']) # Compiled source code
Coops_interface = Coops_compiled_sol['<stdin>:Coops']

coops = w3.eth.contract(
    address = '0x3F5C926EC1BbDBD6C54DBdAAde5573D8e0205121',
    abi = Coops_interface['abi'],
)

## Repartir Coops

In [32]:
i_addresses = 5
acct = w3.eth.account.privateKeyToAccount(private_keys[0])
balance = int(coops.call().balanceOf(public_keys[0])/(i_addresses))
for i in range(1,i_addresses+1):
    address = public_keys[i]
    i_balance = int(coops.call().balanceOf(public_keys[i]))
    if i_balance == 0: 
        construct_txn = coops.functions.transfer(address,balance).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gas': 3000000,
            'gasPrice': w3.toWei('1.4', 'gwei')})
        signed = acct.signTransaction(construct_txn)

        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        # Wait for the transaction to be mined, and get the transaction receipt
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print('Balance of '+address+' = '+str(coops.call().balanceOf(address))+ '  Coops')

Balance of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 20000000  Coops
Balance of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 20000000  Coops
Balance of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 20000000  Coops
Balance of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 20000000  Coops
Balance of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 20000000  Coops


## Devolver Coops

In [31]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    balance = coops.call().balanceOf(address)
    if balance > 0:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = coops.functions.transfer(public_keys[0],balance).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gas': 3000000,
            'gasPrice': w3.toWei('1.4', 'gwei')})

        signed = acct.signTransaction(construct_txn)

        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        # Wait for the transaction to be mined, and get the transaction receipt
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        
    print('Balance of '+address+' = '+str(coops.call().balanceOf(address))+ '  Coops')



Balance of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 0  Coops
Balance of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 0  Coops
Balance of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 0  Coops
Balance of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 0  Coops
Balance of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 0  Coops
Balance of 0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B = 0  Coops
Balance of 0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A = 0  Coops
Balance of 0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5 = 0  Coops
Balance of 0xc30B42346A4d15AAdfA44cBf885E758dbe110b86 = 0  Coops
Balance of 0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb = 0  Coops
Balance of 0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F = 0  Coops
Balance of 0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4 = 0  Coops
Balance of 0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0 = 0  Coops
Balance of 0x13079510D41706bc7880d7C20Df1a5AaE3A26A25 = 0  Coops
Balance of 0xE793eF9f5D743F0Bb959144A34de76cD0b8eF6B4 = 0  Coops
Balance of 0xbd10542D544C

In [30]:
tx_receipt.gasUsed

36470